In [1]:
import pythran
import numpy as np
import numba as nb
%load_ext pythran.magic

In [2]:
%%pythran -fopenmp -DUSE_BOOST_SIMD -march=native
import numpy as np
#pythran export pairDistsP(float, float[:,:], float[:,:])
def pairDistsP(sqFluct, evi, evj):
    #n_e = evals.shape[0]
    #cij = 0
    # for n in range(n_e):
    #     l = evals[n]
    #     cij += 1 / l * np.dot(evi[:,n], evj[:,n])
    cij = np.sum(evi*evj)
    d = sqFluct - 2*cij
    return d

#pythran export clusterFluctsP( float[:,:,:], float[:])
def clusterFluctsP(evecs, sqFlucts):
    c_size = evecs.shape[0]
    flucts = np.zeros(c_size)
    for i in range(c_size):
        evi = evecs[i,:,:]
        for j in range(c_size):
            if i==j:
                continue
            evj = evecs[j,:,:]
            sqFluct = sqFlucts[i] + sqFlucts[j]
            flucts[i] += np.sqrt(pairDistsP(sqFluct, evi, evj))
    print(flucts)
    return flucts/(2*c_size)

#pythran export fastFlucts(float[:], float[:,:], int)
def fastFlucts(evals, evecs, n_modes):
    n_atoms = evecs.shape[0]
    flucts = np.zeros(n_atoms)
    for i in range(n_modes):
        flucts += 1/evals[i]*evecs[:,i]**2
    return flucts


C:\Users\colin\miniconda3\envs\odep\lib\site-packages\setuptools\_distutils\_msvccompiler.py:535: UserWarning: Fallback spawn triggered. Please update distutils monkeypatch.
  warnings.warn(


In [3]:
@nb.njit()
def tosqfluct(vec):
    return np.reshape(vec, (-1,3)).sum(axis=-1)

def loopFluctsP(evals, evecs, labels, sqFlucts):
    n_clusters = np.max(labels)+1
    fullRigidities = np.zeros_like(sqFlucts)
    rigidities = np.zeros_like(sqFlucts)
    mobilities = np.zeros_like(sqFlucts)
    n_evals = evals.shape[0]
    evecs = evecs*1/np.sqrt(evals)
    evecs = evecs.reshape(-1,3,n_evals)
    for i in range(n_clusters):
        mask = (labels == i)
        if not np.any(mask):
            print('Some clusters unassigned')
            rigidities[i] += 0
        else:
            cVecs = evecs[mask, :].copy()
            cFlucts = sqFlucts[mask].copy()
            flucts = clusterFluctsP(cVecs, cFlucts)
            totalFlucts = flucts.mean()
            mobility = np.mean(np.sqrt(cFlucts))
            rigidities[mask] = totalFlucts
            fullRigidities[mask] = flucts
            mobilities[mask] = mobility
    return rigidities, fullRigidities, mobilities



def realFlucts(nc, pdb):
    #from input import pdb
    from make_model import getPDB
    capsid, calphas, title = getPDB(pdb)
    
    nModes = 120


    modes = np.load('../results/models/' + pdb + 'anmmodes' + '.npz')
    print(list(modes.keys()))
    evals = modes['evals'][:nModes].copy()
    evecs = modes['evecs'][:,:nModes].copy()
    print(evecs.shape)
    results = np.load('../results/subdivisions/' + pdb + '/' + pdb + '_' + str(nc) + '_results.npz')
    labels = results['labels']
    
    sqFlucts = fastFlucts(evals, evecs, nModes)
    sqFlucts = tosqfluct(sqFlucts)
    
    rigidities, fullRigidities, mobility = loopFluctsP(evals,evecs.copy(),labels,sqFlucts)


    from prody import loadAtoms, writePDB, extendModel, extendAtomicData, ANM, showProtein
    print(rigidities)
    calphas = capsid.select('calpha')
    calphas.setBetas(fullRigidities)
    print(mobility)
    writePDB('../results/subdivisions/' + pdb + '/' + pdb + '_' + str(nc) + '_rigidtest.pdb', calphas, beta=mobility,
             hybrid36=True)
    return rigidities, fullRigidities, mobility, calphas

In [4]:
pdb = '2e0z'
nc = 32
rigidities, fullRigidities, mobility, calphas = realFlucts(nc, pdb)

@> 5488 atoms and 1 coordinate set(s) were parsed in 0.04s.


5488 atoms and 1 coordinate set(s) were parsed in 0.04s.


@> Biomolecular transformations were applied to the coordinate data.


Biomolecular transformations were applied to the coordinate data.
Number Of Residues:  42000
['evals', 'evecs']
(126000, 120)
[0.1321394  0.1321394  0.1321394  ... 0.12173674 0.12173674 0.12173674]
[0.27529301 0.27529301 0.27529301 ... 0.2673653  0.2673653  0.2673653 ]


In [5]:
from make_model import getPDB
from prody import writePDB
capsid, calphas, title = getPDB(pdb)
data = np.load('../results/subdivisions/' + pdb + '_sqFlucts.npz')
sqFlucts = data['sqFlucts']
k = data['k']
sqFlucts = sqFlucts/k
calphas.setBetas(fullRigidities)
writePDB('../results/subdivisions/' + pdb + '/' + pdb + '_' + str(nc) + '_rigidtest.pdb', calphas, beta=fullRigidities,
             hybrid36=True)

@> 5488 atoms and 1 coordinate set(s) were parsed in 0.04s.


5488 atoms and 1 coordinate set(s) were parsed in 0.04s.


@> Biomolecular transformations were applied to the coordinate data.


Biomolecular transformations were applied to the coordinate data.
Number Of Residues:  42000


'../results/subdivisions/2e0z/2e0z_32_rigidtest.pdb'

In [6]:
%%pythran -fopenmp -DUSE_BOOST_SIMD -march=native
import numpy as np
#pythran export pairDistsG(float[:], float[:], float)
def pairDistsG(evi, evj, sq):
    cij = np.sum(evi*evj)
    d = sq - 2*cij
    return d

#pythran export clusterFluctsG(float[:,:], float[:])
def clusterFluctsG(evecs, cFlucts):
    c_size = evecs.shape[0]
    flucts = np.zeros(c_size)
    for i in range(c_size):

        for j in range(c_size):
            if i==j:
                continue
            evi = evecs[i,:]
            evj = evecs[j,:]
            sq = cFlucts[i] + cFlucts[j]
            flucts[i] += np.sqrt(pairDistsG(evi, evj, sq))
    return flucts/(2*c_size)

#pythran export fastFluctsG(float[:,:])
def fastFluctsG(evecs):
    return np.sum(evecs**2,axis=1)




C:\Users\colin\miniconda3\envs\odep\lib\site-packages\setuptools\_distutils\_msvccompiler.py:535: UserWarning: Fallback spawn triggered. Please update distutils monkeypatch.
  warnings.warn(


In [17]:
def loopFluctsG(evals, evecs, labels):
    n_clusters = np.max(labels)+1
    natoms = evecs.shape[0]
    fullRigidities = np.zeros(natoms)
    evecs = evecs*1/np.sqrt(evals)
    rigidities = np.zeros_like(fullRigidities)
    mobilities = np.zeros_like(fullRigidities)
    n_evals = evals.shape[0]
    sqFlucts = fastFluctsG(evecs)
    for i in range(n_clusters):
        mask = (labels == i)
        if not np.any(mask):
            print('Some clusters unassigned')
            rigidities[i] += 0
        else:
            cVecs = evecs[mask, :].copy()
            cFlucts = sqFlucts[mask].copy()
            flucts = clusterFluctsG(cVecs, cFlucts)
            totalFlucts = flucts.mean()
            mobility = np.mean(np.sqrt(cFlucts))
            rigidities[mask] = totalFlucts
            fullRigidities[mask] = flucts
            mobilities[mask] = mobility
    return rigidities, fullRigidities, mobilities

def realFluctsG(nc, pdb):
    model = 'gnm'
    #from input import pdb
    from make_model import getPDB
    capsid, calphas, title = getPDB(pdb)
    
    nModes = 700


    modes = np.load('../results/models/' + pdb + model + 'modes' + '.npz')
    print(list(modes.keys()))
    evals = modes['evals'][:nModes].copy()
    evecs = modes['evecs'][:,:nModes].copy()
    print(evecs.shape)
    results = np.load('../results/subdivisions/' + pdb + '/' + pdb + '_' + str(nc) + '_results.npz')
    labels = results['labels']
    
    sqFlucts = fastFlucts(evals, evecs, nModes)
    
    rigidities, fullRigidities, mobility = loopFluctsG(evals,evecs.copy(),labels)


    from prody import loadAtoms, writePDB, extendModel, extendAtomicData, ANM, showProtein
    print(rigidities)
    calphas = capsid.select('calpha')
    calphas.setBetas(fullRigidities)
    print(mobility)
    writePDB('../results/subdivisions/' + pdb + '/' + pdb + '_' + str(nc) + '_rigidtest.pdb', calphas, beta=rigidities,
             hybrid36=True)
    return rigidities, fullRigidities, mobility, calphas

In [18]:
pdb = '3j4u'
nc = 210
rigidities, fullRigidities, mobility, calphas = realFluctsG(nc, pdb)

@> 24653 atoms and 1 coordinate set(s) were parsed in 0.19s.


24653 atoms and 1 coordinate set(s) were parsed in 0.19s.


@> Biomolecular transformations were applied to the coordinate data.


Biomolecular transformations were applied to the coordinate data.
Number Of Residues:  195240


@> WARNING Indices are exceeding 99999 and hybrid36 format is being used
WARNING Indices are exceeding 99999 and hybrid36 format is being used


['evals', 'evecs']
(195240, 700)
[0.10411358 0.10411358 0.10411358 ... 0.10400867 0.1055001  0.1055001 ]
[0.25564563 0.25564563 0.25564563 ... 0.26575611 0.26751016 0.26751016]


@> WARNING Indices are exceeding 99999 and hybrid36 format is being used
WARNING Indices are exceeding 99999 and hybrid36 format is being used


In [19]:
from make_model import getPDB
from prody import writePDB
capsid, calphas, title = getPDB(pdb)
data = np.load('../results/subdivisions/' + pdb + '_sqFlucts.npz')
sqFlucts = data['sqFlucts']
k = data['k']
sqFlucts = sqFlucts/k
calphas.setBetas(fullRigidities)
writePDB('../results/subdivisions/' + pdb + '/' + pdb + '_' + str(nc) + '_rigidtest.pdb', calphas, beta=fullRigidities,
             hybrid36=True)

@> 24653 atoms and 1 coordinate set(s) were parsed in 0.16s.


24653 atoms and 1 coordinate set(s) were parsed in 0.16s.


@> Biomolecular transformations were applied to the coordinate data.


Biomolecular transformations were applied to the coordinate data.
Number Of Residues:  195240


@> WARNING Indices are exceeding 99999 and hybrid36 format is being used
WARNING Indices are exceeding 99999 and hybrid36 format is being used
@> WARNING Indices are exceeding 99999 and hybrid36 format is being used
WARNING Indices are exceeding 99999 and hybrid36 format is being used


'../results/subdivisions/3j4u/3j4u_210_rigidtest.pdb'

In [ ]:

import py3Dmol
from prody import showProtein, view3D
#showProtein(capsid, flucts=fullRigidities)
view = view3D(calphas)
view.setStyle({'sphere':{'colorscheme':{'prop':'b'}}})


view.show()

In [ ]:

with open('../results/subdivisions/' + pdb + '/' + pdb + '_' + str(nc) + '_rigidtest.pdb') as ifile:
    system = "".join([x for x in ifile])

In [ ]:
import py3Dmol
view = py3Dmol.view(width=800, height=800)
view.addModelsAsFrames(system)
view.setStyle({'sphere':{'colorscheme':{'prop':'b'}}})
view.addSurface(py3Dmol.VDW,{'opacity':0.7,'colorscheme':{'prop':'b','gradient':'sinebow','min':0,'max':70}})
view.zoomTo()
view.show()

In [ ]:
%timeit realFluctsG(4, '2qi9')

@> 10692 atoms and 1 coordinate set(s) were parsed in 0.08s.


10692 atoms and 1 coordinate set(s) were parsed in 0.08s.


@> Biomolecular transformations were applied to the coordinate data.


Biomolecular transformations were applied to the coordinate data.
Number Of Residues:  1389
['evals', 'evecs']
(1389, 10)
go


In [ ]:
%timeit realFlucts(4, '2qi9')

In [ ]:
pdb = '2qi9'
nc = 4
data = np.load('../results/subdivisions/' + pdb + '_sqFlucts.npz')
sqFlucts = data['sqFlucts']
modes = np.load('../results/models/' + pdb + 'anm' + 'modes.npz')
evals = modes['evals']
evecs = modes['evecs']
results = np.load('../results/subdivisions/' + pdb + '/' + pdb + '_' + str(nc) + '_results.npz')
labels = results['labels']

In [ ]:
mask = (labels == 2)
evecs = evecs.reshape(-1,3,evals.shape[0])
cVecs = evecs[mask, :]
# cVecs = cVecs.reshape(-1, n_evals)
cFlucts = sqFlucts[mask]


In [ ]:
c_size = evecs.shape[0]
flucts = np.zeros(c_size)
i = 5
j = 7
evi = cVecs[i,:,:]
evj = cVecs[j,:,:]
sqFluct = cFlucts[i] + cFlucts[j]

In [ ]:
%load_ext line_profiler
#%load_ext autoreload
#%autoreload 2

In [ ]:
%lprun -f clusterFlucts clusterFlucts(evals, cVecs, cFlucts)

In [1]:
#from eval import realFlucts
import numba as nb
import numpy as np

@nb.njit(nb.float64(nb.float64, nb.float64[:], nb.float64[:,:],nb.float64[:,:]), fastmath=False, nogil=True)
def pairDists(sqFluct, evals, evi, evj):
    n_e = evals.shape[0]
    cij = 0
    for n in range(n_e):
        l = evals[n]
        cij += 1 / l * np.sum(evi[:,n] * evj[:,n])
    d = sqFluct - 2*cij
    return d

@nb.njit(nb.float64[:](nb.float64[:],nb.float64[:,:,:],nb.float64[:]),nogil=True)
def clusterFlucts(evals, evecs, sqFlucts):
    c_size = evecs.shape[0]
    flucts = np.zeros(c_size)
    for i in range(c_size):
        for j in range(c_size):
            if i==j:
                continue
            evi = evecs[i,:,:]
            evj = evecs[j,:,:]
            sqFluct = sqFlucts[i] + sqFlucts[j]
            flucts[i] += pairDists(sqFluct, evals, evi, evj)/2
    return flucts

@nb.njit(nogil=True)
def loopFlucts(evals, evecs, labels, sqFlucts):
    n_clusters = np.max(labels)+1
    fullRigidities = np.zeros_like(sqFlucts)
    rigidities = np.zeros_like(sqFlucts)
    n_evals = evals.shape[0]
    evecs = evecs.reshape(-1,3,n_evals)
    for i in range(n_clusters):
        mask = (labels == i)
        if not np.any(mask):
            print('Some clusters unassigned')
            rigidities[i] = 0
        else:
            cVecs = evecs[mask, :].copy()
            # cVecs = cVecs.reshape(-1, n_evals)
            cFlucts = sqFlucts[mask].copy()
            flucts = clusterFlucts(evals, cVecs, cFlucts)
            totalFlucts = flucts.sum()
            fullRigidities[mask] = flucts
            rigidities[mask] = totalFlucts

    return rigidities, fullRigidities